# Preparation

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
print("Train shape", train.shape )
train.head()

Train shape (750000, 18)


,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [2]:
orig = pd.read_csv('/kaggle/input/bank-marketing-dataset-full/bank-full.csv', sep=';')
orig['y'] = orig['y'].map({'no': 0, 'yes': 1})
orig.columns = train.columns.drop(['id'])
orig.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


In [3]:
test = pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")
print("Test shape", test.shape )
test.head()

Test shape (250000, 17)


,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [4]:
# Store ids and targets
train_ids = train['id']
test_ids = test['id']
y_train = train['y']
y_orig = orig['y']

In [5]:
train = train.drop(columns=['id', 'y'])
test = test.drop(columns=['id'])
orig = orig.drop(columns=['y'])

In [6]:
target_train = pd.concat([y_train, y_orig], ignore_index=True)

In [7]:
all_data = pd.concat([train, test, orig], ignore_index=True)

In [8]:
train = pd.concat([train, orig], axis = 0, ignore_index=True)

In [9]:
RMV = ['y','id']
FEATURES = [c for c in train.columns if not c in RMV]
print("Our features are:")
print( FEATURES )

Our features are:
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']


In [10]:
NUM_FEATURES = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
CAT_FEATURES = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
ohe = OneHotEncoder(handle_unknown="ignore",sparse_output=False)

# Fit on training categorical features and transform both train & test
ohe_train = ohe.fit_transform(train[CAT_FEATURES])
ohe_test = ohe.transform(test[CAT_FEATURES])

# Convert back to DataFrame with proper column names
ohe_cols = ohe.get_feature_names_out(CAT_FEATURES)
train_ohe_df = pd.DataFrame(ohe_train, columns=ohe_cols, index=train.index)
test_ohe_df = pd.DataFrame(ohe_test, columns=ohe_cols, index=test.index)

# Drop original categorical columns and add encoded ones
train = pd.concat([train.drop(columns=CAT_FEATURES), train_ohe_df], axis=1)
test = pd.concat([test.drop(columns=CAT_FEATURES), test_ohe_df], axis=1)

FEATURES = [c for c in train.columns if not c in RMV]

#le = LabelEncoder()
#for feature in CAT_FEATURES:
#    train[feature] = le.fit_transform(train[feature])
#    test[feature] = le.transform(test[feature])

In [11]:
from sklearn.model_selection import StratifiedKFold
from cuml.neighbors import KNeighborsClassifier

In [12]:
# WEIGHTS TO ADJUST IMPORTANCE OF FEATURES DURING KNN
WGT = {'age': 1, 'job': 1, 'marital': 1, 'education': 1, 'default': 1, 'balance': 1, 'housing': 1, 
       'loan': 1, 'contact': 1, 'day': 1, 'month': 1, 'duration': 1, 'campaign': 1, 'pdays': 1, 'previous': 1, 'poutcome': 1}

# KNN Training

In [13]:
train.shape

(795211, 51)

In [14]:
from xgboost import XGBRegressor, XGBClassifier
import xgboost
print("Using XGBoost version",xgboost.__version__)

from catboost import CatBoostClassifier

Using XGBoost version 2.0.3


In [15]:
%%time
FOLDS = 5
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=777)
    
oof_catb = np.zeros(len(train))
pred_catb = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(skf.split(train[FEATURES], target_train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)

    x_train = train.loc[train_index,FEATURES].copy()
    y_train = target_train.loc[train_index].copy()
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = target_train.loc[test_index].copy()
    x_test = test[FEATURES].copy()

    
    for c in NUM_FEATURES:
        m = x_train[c].mean()
        s = x_train[c].std()
        x_train[c] = WGT[c] * (x_train[c]-m)/s
        x_valid[c] = WGT[c] * (x_valid[c]-m)/s
        x_test[c] = WGT[c] * (x_test[c]-m)/s
        x_test[c] = x_test[c].fillna(0)
        x_train[c] = x_train[c].fillna(0)


    model = CatBoostClassifier(
        iterations=10_000,
        depth=5,
        learning_rate=0.0377,
        l2_leaf_reg=2.7384616507314177,
        eval_metric="AUC",
        random_seed=777,
        bootstrap_type = 'Bernoulli',
        subsample = 0.8111694121602908,
        task_type="GPU",  # or "CPU" if GPU not available
        early_stopping_rounds=100,
        verbose=100
    )
    model.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],  
    )
    
    # INFER OOF
    oof_catb[test_index] = model.predict_proba(x_valid.values)[:,1]
    # INFER TEST
    pred_catb += model.predict_proba(x_test.values)[:,1]

# COMPUTE AVERAGE TEST PREDS
pred_catb /= FOLDS

#########################
### Fold 1
#########################


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9093237	best: 0.9093237 (0)	total: 240ms	remaining: 40m 4s
100:	test: 0.9484165	best: 0.9484165 (100)	total: 1.31s	remaining: 2m 8s
200:	test: 0.9549172	best: 0.9549172 (200)	total: 2.22s	remaining: 1m 48s
300:	test: 0.9576049	best: 0.9576049 (300)	total: 3.12s	remaining: 1m 40s
400:	test: 0.9589185	best: 0.9589185 (400)	total: 3.98s	remaining: 1m 35s
500:	test: 0.9599193	best: 0.9599193 (500)	total: 4.76s	remaining: 1m 30s
600:	test: 0.9606162	best: 0.9606162 (600)	total: 5.54s	remaining: 1m 26s
700:	test: 0.9611266	best: 0.9611266 (700)	total: 6.32s	remaining: 1m 23s
800:	test: 0.9615189	best: 0.9615189 (800)	total: 7.1s	remaining: 1m 21s
900:	test: 0.9619207	best: 0.9619207 (900)	total: 7.88s	remaining: 1m 19s
1000:	test: 0.9622126	best: 0.9622127 (999)	total: 8.65s	remaining: 1m 17s
1100:	test: 0.9624674	best: 0.9624674 (1100)	total: 9.43s	remaining: 1m 16s
1200:	test: 0.9626905	best: 0.9626905 (1200)	total: 10.2s	remaining: 1m 14s
1300:	test: 0.9628726	best: 0.9628726 (

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9080428	best: 0.9080428 (0)	total: 14.3ms	remaining: 2m 22s
100:	test: 0.9481225	best: 0.9481225 (100)	total: 1.05s	remaining: 1m 43s
200:	test: 0.9546164	best: 0.9546164 (200)	total: 1.93s	remaining: 1m 34s
300:	test: 0.9572961	best: 0.9572961 (300)	total: 2.8s	remaining: 1m 30s
400:	test: 0.9587243	best: 0.9587243 (400)	total: 3.66s	remaining: 1m 27s
500:	test: 0.9595972	best: 0.9595972 (500)	total: 4.53s	remaining: 1m 25s
600:	test: 0.9602848	best: 0.9602848 (600)	total: 5.4s	remaining: 1m 24s
700:	test: 0.9608217	best: 0.9608217 (700)	total: 6.24s	remaining: 1m 22s
800:	test: 0.9612214	best: 0.9612214 (800)	total: 7.09s	remaining: 1m 21s
900:	test: 0.9615586	best: 0.9615594 (899)	total: 7.93s	remaining: 1m 20s
1000:	test: 0.9618285	best: 0.9618285 (1000)	total: 8.76s	remaining: 1m 18s
1100:	test: 0.9620731	best: 0.9620731 (1100)	total: 9.59s	remaining: 1m 17s
1200:	test: 0.9622892	best: 0.9622892 (1200)	total: 10.4s	remaining: 1m 16s
1300:	test: 0.9624700	best: 0.9624700

...

0:	test: 0.9087122	best: 0.9087122 (0)	total: 14.6ms	remaining: 2m 25s
100:	test: 0.9478951	best: 0.9478951 (100)	total: 1.06s	remaining: 1m 43s
200:	test: 0.9541252	best: 0.9541252 (200)	total: 1.97s	remaining: 1m 36s
300:	test: 0.9568738	best: 0.9568738 (300)	total: 2.9s	remaining: 1m 33s
400:	test: 0.9583672	best: 0.9583672 (400)	total: 3.79s	remaining: 1m 30s
500:	test: 0.9594218	best: 0.9594218 (500)	total: 4.66s	remaining: 1m 28s
600:	test: 0.9600377	best: 0.9600377 (600)	total: 5.53s	remaining: 1m 26s
700:	test: 0.9605279	best: 0.9605279 (700)	total: 6.38s	remaining: 1m 24s
800:	test: 0.9609059	best: 0.9609059 (800)	total: 7.22s	remaining: 1m 22s
900:	test: 0.9611920	best: 0.9611920 (900)	total: 8.06s	remaining: 1m 21s
1000:	test: 0.9614865	best: 0.9614865 (1000)	total: 8.92s	remaining: 1m 20s
1100:	test: 0.9617050	best: 0.9617050 (1100)	total: 9.77s	remaining: 1m 18s
1200:	test: 0.9619063	best: 0.9619063 (1200)	total: 10.6s	remaining: 1m 17s
1300:	test: 0.9620773	best: 0.962077

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9096651	best: 0.9096651 (0)	total: 14.6ms	remaining: 2m 25s
100:	test: 0.9481827	best: 0.9481827 (100)	total: 1.03s	remaining: 1m 41s
200:	test: 0.9547130	best: 0.9547130 (200)	total: 1.94s	remaining: 1m 34s
300:	test: 0.9575695	best: 0.9575695 (300)	total: 2.79s	remaining: 1m 30s
400:	test: 0.9590819	best: 0.9590819 (400)	total: 3.64s	remaining: 1m 27s
500:	test: 0.9600663	best: 0.9600663 (500)	total: 4.47s	remaining: 1m 24s
600:	test: 0.9606977	best: 0.9606977 (600)	total: 5.3s	remaining: 1m 22s
700:	test: 0.9611753	best: 0.9611753 (700)	total: 6.13s	remaining: 1m 21s
800:	test: 0.9616033	best: 0.9616033 (800)	total: 6.96s	remaining: 1m 19s
900:	test: 0.9619563	best: 0.9619563 (900)	total: 7.79s	remaining: 1m 18s
1000:	test: 0.9622234	best: 0.9622234 (1000)	total: 8.62s	remaining: 1m 17s
1100:	test: 0.9624677	best: 0.9624677 (1100)	total: 9.45s	remaining: 1m 16s
1200:	test: 0.9626424	best: 0.9626424 (1200)	total: 10.3s	remaining: 1m 15s
1300:	test: 0.9628431	best: 0.962843

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9103354	best: 0.9103354 (0)	total: 13.7ms	remaining: 2m 16s
100:	test: 0.9471900	best: 0.9471900 (100)	total: 986ms	remaining: 1m 36s
200:	test: 0.9538628	best: 0.9538628 (200)	total: 1.83s	remaining: 1m 29s
300:	test: 0.9564790	best: 0.9564790 (300)	total: 2.66s	remaining: 1m 25s
400:	test: 0.9579840	best: 0.9579840 (400)	total: 3.49s	remaining: 1m 23s
500:	test: 0.9589307	best: 0.9589307 (500)	total: 4.34s	remaining: 1m 22s
600:	test: 0.9596193	best: 0.9596193 (600)	total: 5.17s	remaining: 1m 20s
700:	test: 0.9601464	best: 0.9601464 (700)	total: 6.05s	remaining: 1m 20s
800:	test: 0.9605194	best: 0.9605194 (800)	total: 6.89s	remaining: 1m 19s
900:	test: 0.9608632	best: 0.9608632 (900)	total: 7.74s	remaining: 1m 18s
1000:	test: 0.9611188	best: 0.9611188 (1000)	total: 8.59s	remaining: 1m 17s
1100:	test: 0.9613711	best: 0.9613711 (1100)	total: 9.43s	remaining: 1m 16s
1200:	test: 0.9615835	best: 0.9615835 (1200)	total: 10.3s	remaining: 1m 15s
1300:	test: 0.9617553	best: 0.96175

In [16]:
from sklearn.metrics import roc_auc_score
true = target_train.values
m = roc_auc_score(true, oof_catb)
print(f"CatBoost CV Score AUC = {m:.3f}")

CatBoost CV Score AUC = 0.965


In [17]:
%%time
FOLDS = 5
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=777)
    
oof_xgb = np.zeros(len(train))
pred_xgb = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(skf.split(train[FEATURES], target_train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)

    x_train = train.loc[train_index,FEATURES].copy()
    y_train = target_train.loc[train_index].copy()
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = target_train.loc[test_index].copy()
    x_test = test[FEATURES].copy()

    
    for c in NUM_FEATURES:
        m = x_train[c].mean()
        s = x_train[c].std()
        x_train[c] = WGT[c] * (x_train[c]-m)/s
        x_valid[c] = WGT[c] * (x_valid[c]-m)/s
        x_test[c] = WGT[c] * (x_test[c]-m)/s
        x_test[c] = x_test[c].fillna(0)
        x_train[c] = x_train[c].fillna(0)

    model = XGBClassifier(
        device="cuda",
        max_depth=6,  
        colsample_bytree=0.9, 
        subsample=0.9, 
        n_estimators=10_000,  
        learning_rate=0.1, 
        eval_metric="auc",
        early_stopping_rounds=100,
        alpha=1
    )
    model.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],
        verbose=100
    )

    # INFER OOF
    oof_xgb[test_index] = model.predict_proba(x_valid.values)[:,1]
    # INFER TEST
    pred_xgb += model.predict_proba(x_test.values)[:,1]

pred_xgb /= FOLDS

#########################
### Fold 1
#########################
[0]	validation_0-auc:0.93086
[100]	validation_0-auc:0.96227
[200]	validation_0-auc:0.96479
[300]	validation_0-auc:0.96568
[400]	validation_0-auc:0.96628
[500]	validation_0-auc:0.96664
[600]	validation_0-auc:0.96692
[700]	validation_0-auc:0.96709
[800]	validation_0-auc:0.96727
[900]	validation_0-auc:0.96737
[1000]	validation_0-auc:0.96741
[1100]	validation_0-auc:0.96744
[1200]	validation_0-auc:0.96745
[1300]	validation_0-auc:0.96747
[1400]	validation_0-auc:0.96742


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [05:20:48] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


#########################
### Fold 2
#########################
[0]	validation_0-auc:0.92932
[100]	validation_0-auc:0.96196
[200]	validation_0-auc:0.96450
[300]	validation_0-auc:0.96560
[400]	validation_0-auc:0.96614
[500]	validation_0-auc:0.96654
[600]	validation_0-auc:0.96679
[700]	validation_0-auc:0.96689
[800]	validation_0-auc:0.96699
[900]	validation_0-auc:0.96706
[1000]	validation_0-auc:0.96708
[1100]	validation_0-auc:0.96712
[1200]	validation_0-auc:0.96714
[1261]	validation_0-auc:0.96710
#########################
### Fold 3
#########################
[0]	validation_0-auc:0.93181
[100]	validation_0-auc:0.96155
[200]	validation_0-auc:0.96383
[300]	validation_0-auc:0.96482
[400]	validation_0-auc:0.96535
[500]	validation_0-auc:0.96570
[600]	validation_0-auc:0.96595
[700]	validation_0-auc:0.96613
[800]	validation_0-auc:0.96622
[900]	validation_0-auc:0.96630
[1000]	validation_0-auc:0.96634
[1100]	validation_0-auc:0.96635
[1200]	validation_0-auc:0.96635
[1300]	validation_0-auc:0.96638
[1

In [18]:
from sklearn.metrics import roc_auc_score
true = target_train.values
m = roc_auc_score(true, oof_xgb)
print(f"XGBoost CV Score AUC = {m:.3f}")

XGBoost CV Score AUC = 0.967


In [19]:
%%time
from lightgbm import LGBMClassifier
import lightgbm as lgb

FOLDS = 5
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=777)

oof_lgbm = np.zeros(len(train))
pred_lgbm = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(skf.split(train[FEATURES], target_train)):
    print("#" * 25)
    print(f"### Fold {i+1}")
    print("#" * 25)

    x_train = train.loc[train_index, FEATURES].copy()
    y_train = target_train.loc[train_index].copy()
    x_valid = train.loc[test_index, FEATURES].copy()
    y_valid = target_train.loc[test_index].copy()
    x_test = test[FEATURES].copy()

    # Standardize numeric features with WGT scaling
    for c in NUM_FEATURES:
        m = x_train[c].mean()
        s = x_train[c].std()
        x_train[c] = WGT[c] * (x_train[c] - m) / s
        x_valid[c] = WGT[c] * (x_valid[c] - m) / s
        x_test[c] = WGT[c] * (x_test[c] - m) / s
        x_test[c] = x_test[c].fillna(0)
        x_train[c] = x_train[c].fillna(0)

    # LightGBM model
    model = LGBMClassifier(
        max_depth=6,
        colsample_bytree=0.9,
        subsample=0.9,
        n_estimators=10_000,
        learning_rate=0.1,
        reg_alpha=1,
        objective="binary",
        metric="auc",
        random_state=777,
        n_jobs=-1
    )

    model.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],
        eval_metric="auc",
        callbacks=[
            # Early stopping with LightGBM's API
            lgb.early_stopping(100),
            lgb.log_evaluation(100)
        ]
    )

    # INFER OOF
    oof_lgbm[test_index] = model.predict_proba(x_valid)[:, 1]
    # INFER TEST
    pred_lgbm += model.predict_proba(x_test)[:, 1]

# Average test predictions across folds
pred_lgbm /= FOLDS

#########################
### Fold 1
#########################
[LightGBM] [Info] Number of positive: 76621, number of negative: 559547
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032228 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1048
[LightGBM] [Info] Number of data points in the train set: 636168, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120441 -> initscore=-1.988256
[LightGBM] [Info] Start training from score -1.988256
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.96192
[200]	valid_0's auc: 0.964388
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[300]	valid_0's auc: 0.965452
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [20]:
from sklearn.metrics import roc_auc_score
true = target_train.values
m = roc_auc_score(true, oof_lgbm)
print(f"LGBM CV Score AUC = {m:.3f}")

LGBM CV Score AUC = 0.967


# Ensembling

In [21]:
oof_ens = 0.5 * oof_lgbm + 0.25 * oof_xgb + 0.25 * oof_catb
m = roc_auc_score(true, oof_ens)
print(f" CV Score AUC = {m:.3f}")

 CV Score AUC = 0.967


In [31]:
best_public = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
best_public = best_public.y.values

best_public

array([0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5])

In [34]:
from scipy.stats import rankdata

sub = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
sub.y = -0.1 * (0.25 * rankdata (pred_lgbm) + 0.25 * rankdata (pred_xgb) + 0.5 * rankdata(pred_catb)) + 1.1 * rankdata(best_public)
sub.y = rankdata( sub.y ) / len(sub)
print( sub.shape )
sub.to_csv(f"submission.csv",index=False)
sub.head()

(250000, 2)


,id,y
0,750000,0.438528
1,750001,0.252316
2,750002,0.899200
3,750003,0.962360
4,750004,0.323116


In [35]:
print("Successfully saved as CSV file")

Successfully saved as CSV file
